In [131]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [132]:
dataset = pd.read_csv("train.csv")
dataset = dataset.drop('Id',axis=1)
dataset.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [133]:
#XGBoost
import xgboost as xgb
xgb_data = dataset

In [134]:
X = xgb_data.drop(columns=['SalePrice'])
y = xgb_data['SalePrice']

In [135]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)
label_encoders = {}
for column in X_train.select_dtypes(include=['object']).columns:
    label_encoders[column]=LabelEncoder()
    X_train[column] = label_encoders[column].fit_transform(X_train[column])
    X_test[column] = label_encoders[column].fit_transform(X_test[column])


In [136]:
xgbmodel = xgb.XGBRegressor()
xgbmodel.fit(X_train,y_train)
xgbpred = xgbmodel.predict(X_test)

In [137]:
from sklearn.metrics import mean_squared_error
xgbmse = mean_squared_error(y_test,xgbpred)
print("XGBoost Mean Squared Error:", xgbmse)


XGBoost Mean Squared Error: 797480696.0504382


In [138]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
rf_data = dataset

In [139]:
print(X_train_final.dtypes)

MSSubClass         int64
LotFrontage      float64
LotArea            int64
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
MasVnrArea       float64
BsmtFinSF1         int64
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
1stFlrSF           int64
2ndFlrSF           int64
LowQualFinSF       int64
GrLivArea          int64
BsmtFullBath       int64
BsmtHalfBath       int64
FullBath           int64
HalfBath           int64
BedroomAbvGr       int64
KitchenAbvGr       int64
TotRmsAbvGrd       int64
Fireplaces         int64
GarageYrBlt      float64
GarageCars         int64
GarageArea         int64
WoodDeckSF         int64
OpenPorchSF        int64
EnclosedPorch      int64
3SsnPorch          int64
ScreenPorch        int64
PoolArea           int64
MiscVal            int64
MoSold             int64
YrSold             int64
0                 object
dtype: object


In [140]:
rf_cat = rf_data.select_dtypes(include=['object'])
cat_columns = rf_data.select_dtypes(include=['object'])
rf_cat_enc = pd.get_dummies(rf_cat,drop_first=True,dtype=int)
rf_cat_enc.head()

,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,1,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
1,0,0,1,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
2,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [141]:
rf_data = rf_data.drop(columns=(cat_columns))
rf_data = pd.concat([rf_data,rf_cat_enc],axis=1)

In [142]:
enc_data = rf_data

In [143]:
X=rf_data.drop(columns=['SalePrice'])
y=rf_data['SalePrice']

In [144]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)
rfmodel = RandomForestRegressor()
rfmodel.fit(X_train,y_train)
rfpreds=rfmodel.predict(X_test)

In [145]:
rfmse = mean_squared_error(y_test,rfpreds)
print("RF Mean Squared Error:", rfmse)


RF Mean Squared Error: 1809642527.7788832


In [146]:
from sklearn.linear_model import LinearRegression
lin_data = enc_data.dropna()
X=lin_data.drop(columns=['SalePrice'])
y=lin_data['SalePrice']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)
linreg = LinearRegression()
linreg.fit(X_train,y_train)
linpreds = linreg.predict(X_test)
linmse = mean_squared_error(y_test,linpreds)
print("LR Mean Squared Error:", linmse)


LR Mean Squared Error: 1957708034.285432


In [147]:
from sklearn.linear_model import ElasticNet
el_data = enc_data.dropna()
X=el_data.drop(columns=['SalePrice'])
y=el_data['SalePrice']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)
el_net = ElasticNet(alpha=0.1,l1_ratio=0.5)
el_net.fit(X_train,y_train)
elpreds = el_net.predict(X_test)
elmse = mean_squared_error(y_test,elpreds)
print("EL Mean Squared Error:", elmse)


EL Mean Squared Error: 1092416181.2961912


c:\Users\benja\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.238e+11, tolerance: 5.413e+08
  model = cd_fast.enet_coordinate_descent(


In [148]:
from sklearn.linear_model import Lasso
la_data = enc_data.dropna()
X=la_data.drop(columns=['SalePrice'])
y=la_data['SalePrice']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)
lasso_mod = Lasso(alpha=0.1)
lasso_mod.fit(X_train,y_train)
lapreds = lasso_mod.predict(X_test)
lamse = mean_squared_error(y_test,lapreds)
print("EL Mean Squared Error:", lamse)


EL Mean Squared Error: 2492480823.0484076


c:\Users\benja\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.759e+11, tolerance: 5.413e+08
  model = cd_fast.enet_coordinate_descent(


In [153]:
from sklearn.neighbors import KNeighborsRegressor
knn_data = enc_data.dropna()
X=knn_data.drop(columns=['SalePrice'])
y=knn_data['SalePrice']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)
knn_mod = KNeighborsRegressor(n_neighbors=5)
knn_mod.fit(X_train,y_train)
knn_preds = knn_mod.predict(X_test)
knnmse = mean_squared_error(y_test,knn_preds)
print("EL Mean Squared Error:", knnmse)


EL Mean Squared Error: 2768118784.4324555


In [152]:
preds = {
    'XGBoost': xgbpred,
    'RandomForest': rfpreds,
    'Linear': linpreds,
    'ElasticNet': elpreds,
    'Lasso': lapreds,
    'KNN': knn_preds
}
preds = pd.DataFrame(preds)
preds.head()

ValueError: All arrays must be of the same length

In [154]:
dataset['SalePrice'].std()

79442.50288288662

In [160]:
#submission stuff
untouched = pd.read_csv("test.csv")
final_data = pd.read_csv("test.csv")
final_data = final_data.drop('Id',axis=1)
label_encoders = {}
for column in final_data.select_dtypes(include=['object']).columns:
    label_encoders[column]=LabelEncoder()
    final_data[column] = label_encoders[column].fit_transform(final_data[column])

final_preds = xgbmodel.predict(final_data)
final_preds = pd.DataFrame(final_preds)
final_preds = pd.concat(untouched['Id'],final_preds)
final_preds.to_csv("final_preds.csv",index=False)
    

TypeError: concat() takes 1 positional argument but 2 were given